In [12]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from keras.layers.merge import concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, BatchNormalization, Dense, Conv2D, MaxPool2D , Flatten, Dropout, GlobalAveragePooling2D, AveragePooling2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
import sys
from PIL import Image
sys.modules['Image'] = Image 

In [3]:
tf.config.list_physical_devices('GPU')

2022-01-10 10:37:39.054601: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-01-10 10:37:39.056783: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-01-10 10:37:39.121944: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-10 10:37:39.122241: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.91GiB deviceMemoryBandwidth: 451.17GiB/s
2022-01-10 10:37:39.122276: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-01-10 10:37:39.127588: I tensorflow/stream_executor/platform/defau

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

rm/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2022-01-10 10:37:39.129001: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcufft.so.10
2022-01-10 10:37:39.130074: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcurand.so.10
2022-01-10 10:37:39.135289: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcusolver.so.10
2022-01-10 10:37:39.136289: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcusparse.so.11
2022-01-10 10:37:39.136637: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8
2022-01-10 10:37:39.136769: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so r

In [4]:
print("GPUs: ", len(tf.config.experimental.list_physical_devices('GPU')))

GPUs:  1


In [5]:
train = pd.read_csv('train_metadata.csv')
test = pd.read_csv('test_metadata.csv')
#submission = pd.read_csv('sample_submission.csv')

train_dir = 'train/'
test_dir = 'test/'

In [6]:
label = []
train_data = []
for i in range(train.shape[0]):
    train_data.append(train_dir + train['image_name'].iloc[i] + '.jpg')
    label.append(train['diagnosis'].iloc[i])
    
test_data = []
for i in range(test.shape[0]):
    test_data.append(test_dir + test['image'].iloc[i] + '.jpg')

In [7]:
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder = OneHotEncoder(sparse=False)
label_df = pd.DataFrame(label, columns = ['diagnosis'])
one_hot_encoder.fit(label_df)

label_df_encoded = one_hot_encoder.transform(label_df)
label_df_encoded = pd.DataFrame(data=label_df_encoded, columns=one_hot_encoder.categories_)
print(label_df_encoded.columns)

MultiIndex([('atypical melanocytic proliferation',),
            (               'cafe-au-lait macule',),
            (                       'lentigo NOS',),
            (               'lichenoid keratosis',),
            (                          'melanoma',),
            (                             'nevus',),
            (              'seborrheic keratosis',),
            (                     'solar lentigo',),
            (                           'unknown',)],
           )


In [8]:
df_train = pd.DataFrame(train_data)
df_train.columns = ['images']
df_train = pd.concat([df_train, label_df_encoded], axis = 1)
print(df_train.columns)
df_test = pd.DataFrame(test_data)
df_test.columns = ['images']

Index([                               'images',
       ('atypical melanocytic proliferation',),
                      ('cafe-au-lait macule',),
                              ('lentigo NOS',),
                      ('lichenoid keratosis',),
                                 ('melanoma',),
                                    ('nevus',),
                     ('seborrheic keratosis',),
                            ('solar lentigo',),
                                  ('unknown',)],
      dtype='object')


In [9]:
X_train, X_test, y_train, y_test = train_test_split(df_train.iloc[:, 0], df_train.iloc[:,1::], test_size = 0.3, random_state = 42)

train = pd.DataFrame(X_train)
train.columns = ['images']
train = pd.concat([train, y_train], axis = 1)

test = pd.DataFrame(X_test)
test.columns = ['images']
test = pd.concat([test, y_test], axis = 1)

In [10]:
train_datagen = ImageDataGenerator(rescale=1./255,rotation_range=50,
                                   zoom_range = 0.3,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range = 0.2,
                                   horizontal_flip=True,
                                   fill_mode = 'nearest')
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    train,
    x_col=train.columns[0],
    y_col=train.columns[1:],
    target_size=(224, 224),
    batch_size=8,
    shuffle=True,
    class_mode='raw')

test_generator = test_datagen.flow_from_dataframe(
    test,
    x_col=test.columns[0],
    y_col=test.columns[1:],
    target_size=(224, 224),
    shuffle=False,
    batch_size=8,
    class_mode='raw')

Found 23188 validated image filenames.
Found 9938 validated image filenames.


In [13]:
model_d=DenseNet201(weights='imagenet',include_top=False, input_shape=(224, 224, 3)) 

x=model_d.output

x= GlobalAveragePooling2D()(x)

x= BatchNormalization()(x)

x= Dropout(0.5)(x)

x= Dense(1024,activation='relu')(x) 

x= Dense(512,activation='relu')(x) 

x= BatchNormalization()(x)

x= Dropout(0.5)(x)

class_num = 9

preds=Dense(class_num,activation='softmax')(x) #FC-layer
model=Model(inputs=model_d.input,outputs=preds)

model.summary()

2022-01-10 10:38:35.286700: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-10 10:38:35.287016: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-01-10 10:38:35.287184: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-10 10:38:35.287809: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.91GiB deviceMemoryBandwidth: 451.17GiB/s
2022-01-10 10:38

74842112/74836368 [==============================] - 26s 0us/step
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
____________________________

In [14]:
for layer in model.layers[:-8]:

    layer.trainable=False

for layer in model.layers[-8:]:

    layer.trainable=True

In [15]:
opt = Adam(lr=0.0001)

model.compile(optimizer= opt,loss='categorical_crossentropy',metrics=['accuracy'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
______________________________________________________________________________________________

In [16]:
batch_size = 128
epochs = 100
nb_train_steps = train.shape[0] // batch_size
nb_val_steps = test.shape[0] // batch_size

checkpoint = ModelCheckpoint("dense-201.h5", monitor='val_accuracy', verbose=5, save_best_only=True, save_weights_only=False, mode='auto')

early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=20, verbose=1, mode='auto')

model.fit_generator(train_generator, steps_per_epoch=nb_train_steps, epochs=epochs,
                              validation_data=test_generator, validation_steps=nb_val_steps, 
                              verbose=1, callbacks=[checkpoint,early])

/home/deeplab/anaconda3/envs/tensorflow/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2022-01-10 10:39:19.325514: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-01-10 10:39:19.342193: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 4200000000 Hz


Epoch 1/100


2022-01-10 10:39:24.902144: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2022-01-10 10:39:25.063049: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2022-01-10 10:39:25.279692: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8


181/181 [==============================] - 95s 487ms/step - loss: 3.1469 - accuracy: 0.1703 - val_loss: 1.6382 - val_accuracy: 0.5633

Epoch 00001: val_accuracy improved from -inf to 0.56331, saving model to dense-201.h5
Epoch 2/100
181/181 [==============================] - 84s 467ms/step - loss: 2.4814 - accuracy: 0.3562 - val_loss: 1.4671 - val_accuracy: 0.6477

Epoch 00002: val_accuracy improved from 0.56331 to 0.64773, saving model to dense-201.h5
Epoch 3/100
181/181 [==============================] - 85s 471ms/step - loss: 1.9343 - accuracy: 0.4653 - val_loss: 1.4067 - val_accuracy: 0.6769

Epoch 00003: val_accuracy improved from 0.64773 to 0.67695, saving model to dense-201.h5
Epoch 4/100
181/181 [==============================] - 85s 469ms/step - loss: 1.7872 - accuracy: 0.4933 - val_loss: 1.0623 - val_accuracy: 0.7744

Epoch 00004: val_accuracy improved from 0.67695 to 0.77435, saving model to dense-201.h5
Epoch 5/100
181/181 [==============================] - 87s 479ms/step -


Epoch 00038: val_accuracy did not improve from 0.91558
Epoch 39/100
181/181 [==============================] - 84s 467ms/step - loss: 0.3728 - accuracy: 0.8900 - val_loss: 0.3754 - val_accuracy: 0.9091

Epoch 00039: val_accuracy did not improve from 0.91558
Epoch 40/100
181/181 [==============================] - 87s 479ms/step - loss: 0.4049 - accuracy: 0.8894 - val_loss: 0.3964 - val_accuracy: 0.8815

Epoch 00040: val_accuracy did not improve from 0.91558
Epoch 41/100
181/181 [==============================] - 86s 475ms/step - loss: 0.4412 - accuracy: 0.8749 - val_loss: 0.4009 - val_accuracy: 0.8815

Epoch 00041: val_accuracy did not improve from 0.91558
Epoch 42/100
181/181 [==============================] - 84s 465ms/step - loss: 0.3769 - accuracy: 0.8902 - val_loss: 0.3902 - val_accuracy: 0.8864

Epoch 00042: val_accuracy did not improve from 0.91558
Epoch 43/100
181/181 [==============================] - 84s 467ms/step - loss: 0.4219 - accuracy: 0.8776 - val_loss: 0.3708 - val_ac